In [1]:
%autosave 0
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline

# Téléchargement de la liste de stop words en français.
nltk.download('stopwords')

# Importations de la liste de stop words en français.
from nltk.corpus import stopwords
stop_words_french = set(stopwords.words('french'))

Autosave disabled


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\micka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Lecture du fichier.
df_test = pd.read_excel("df_clean.xlsx")
df_test.head(10)

,Client,Commentaire,Note
0,marc lesterpt,Je commande mes pneus de motos et de voitures ...,5
1,Séverine M,"BonjourLe site est génial, très bien expliqué ...",5
2,PINTO NUNO,"Tout était parfait, timing parfait dans les da...",5
3,client,Pneus d'un tres bon rapport qualité prix. Les ...,5
4,Vignaux,"Bon suivi de la commande,les questions sont im...",4
5,cliente juliema,Ce service sur place est d un grand secours. R...,5
6,cliente,La commande sur internet c'est très bien passé...,5
7,Manu,"RAS, le rapport qualité prix est très correct....",5
8,Xavier,Fidèle depuis de nombreuses années j aï toujou...,5
9,Sonia Nutrition,M'a été recommandé. Meilleur rapport qualité p...,5


In [3]:
# Création de la colonne "Sentiment" à partir de la colonne "Note".
df_test['Sentiment'] = df_test['Note'].apply(lambda x: 'Négatif' if x in [1, 2] else 'Positif')

In [4]:
# Suppression des caractères spéciaux, chiffres, valeurs manquantes et texte en minuscules.
df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()
df_test['Commentaire'] = df_test['Commentaire'].fillna('') 
df_test.head()

C:\Users\micka\AppData\Local\Temp\ipykernel_12356\311697523.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()


,Client,Commentaire,Note,Sentiment
0,marc lesterpt,je commande mes pneus de motos et de voitures ...,5,Positif
1,Séverine M,bonjourle site est gnial trs bien expliqu pour...,5,Positif
2,PINTO NUNO,tout tait parfait timing parfait dans les date...,5,Positif
3,client,pneus dun tres bon rapport qualit prix les mei...,5,Positif
4,Vignaux,bon suivi de la commandeles questions sont imm...,4,Positif


In [5]:
# Division du DataFrame en un ensemble d'entraînement et un ensemble de test.
X_train, X_test, y_train, y_test = train_test_split(df_test['Commentaire'], df_test['Sentiment'], test_size=0.2, random_state=42)

In [6]:
# Application d'une vectorisation de texte via CountVectorizer avec stop words en français.
vectorizer = CountVectorizer(stop_words=list(stop_words_french))
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [7]:
# Conversion des étiquettes en entiers avec LabelEncoder.
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [8]:
# Création d'un classificateur XGBoost.
clf = XGBClassifier()

In [9]:
# Entrainement du classificateur sur l'ensemble d'entraînement.
clf.fit(X_train, y_train_encoded)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [10]:
# Définition de la grille des hyperparamètres à rechercher.
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [50, 100, 200],}

In [11]:
# Utilisation de GridSearchCV pour la recherche des meilleurs paramètres.
grid_search = GridSearchCV(clf, param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train_encoded)

Fitting 3 folds for each of 27 candidates, totalling 81 fits


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.1, 0.01, 0.05],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [50, 100, 200]},
             verbose=2)

In [12]:
# Identification des meilleurs paramètres.
best_params = grid_search.best_params_
print("Meilleurs paramètres :", best_params)

Meilleurs paramètres : {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}


In [14]:
# Effectuer des prédictions sur l'ensemble de test à partir du meilleur modèle.
best_clf = grid_search.best_estimator_
y_pred_encoded = best_clf.predict(X_test)

In [15]:
# Convertion des prédictions en labels originaux.
y_pred = label_encoder.inverse_transform(y_pred_encoded)

In [16]:
# Affichage du rapport de classification du modèle XGBoost.
print("Rapport de classification :\n", classification_report(y_test, y_pred))

Rapport de classification :
               precision    recall  f1-score   support

     Négatif       0.85      0.63      0.73      7296
     Positif       0.93      0.98      0.96     39181

    accuracy                           0.92     46477
   macro avg       0.89      0.81      0.84     46477
weighted avg       0.92      0.92      0.92     46477



In [17]:
# Calcul et affichage de la matrice de confusion.
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
print("Matrice de confusion :\n", confusion_matrix)

Matrice de confusion :
 Classe prédite  Négatif  Positif
Classe réelle                   
Négatif            4615     2681
Positif             820    38361
